# Start and Test Rekognition Model

## Upgrade awscli and boto3 libraries

In [1]:
!pip install boto3 --upgrade --user

    100% |████████████████████████████████| 133kB 9.2MB/s ta 0:00:011
Requirement not upgraded as not directly required: s3transfer<0.4.0,>=0.3.0 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from boto3) (0.3.3)
Requirement not upgraded as not directly required: jmespath<1.0.0,>=0.7.1 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from boto3) (0.9.4)
    100% |████████████████████████████████| 5.9MB 8.4MB/s eta 0:00:01
Requirement not upgraded as not directly required: python-dateutil<3.0.0,>=2.1 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from botocore<1.16.0,>=1.15.2->boto3) (2.7.3)
Requirement not upgraded as not directly required: urllib3<1.26,>=1.20; python_version != "3.4" in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from botocore<1.16.0,>=1.15.2->boto3) (1.23)
Requirement not upgraded as not directly required: docutils<0.16,>=0.10 in /home/ec2-user/anaconda3/envs/python3/lib/python3

In [2]:
!pip install awscli --upgrade --user

    100% |████████████████████████████████| 2.9MB 13.7MB/s ta 0:00:01
Requirement not upgraded as not directly required: PyYAML<5.3,>=3.10 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from awscli) (5.2)
Requirement not upgraded as not directly required: botocore==1.15.2 in /home/ec2-user/.local/lib/python3.6/site-packages (from awscli) (1.15.2)
Requirement not upgraded as not directly required: colorama<0.4.4,>=0.2.5; python_version != "3.4" in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from awscli) (0.3.9)
Requirement not upgraded as not directly required: docutils<0.16,>=0.10 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from awscli) (0.14)
Requirement not upgraded as not directly required: s3transfer<0.4.0,>=0.3.0 in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (from awscli) (0.3.3)
Requirement not upgraded as not directly required: rsa<=3.5.0,>=3.1.2 in /home/ec2-user/anaconda3/envs/python

## 1- Start your Rekognition model (Editing required)

In [3]:
!aws rekognition start-project-version \
  --project-version-arn "arn:aws:rekognition:us-east-1:283029692235:project/test1/version/test1.2020-02-18T13.43.28/1582055008615" \
  --min-inference-units 1 \
  --endpoint-url https://rekognition.us-east-1.amazonaws.com \
  --region us-east-1

{
    "Status": "STARTING"
}


## 2- Define inference function (No editing required)

In [ ]:
confidence =98
import boto3
import io
%matplotlib inline
from PIL import Image, ImageDraw, ExifTags, ImageColor, ImageFont

def show_custom_labels(model,bucket,photo):
     

    client=boto3.client('rekognition')

    # Load image from S3 bucket
    s3_connection = boto3.resource('s3')

    s3_object = s3_connection.Object(bucket,photo)
    s3_response = s3_object.get()

    stream = io.BytesIO(s3_response['Body'].read())
    image=Image.open(stream)
    
    #Call DetectCustomLabels 
    response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
        MinConfidence=1,
        ProjectVersionArn=model)
   
   
    imgWidth, imgHeight = image.size  
    draw = ImageDraw.Draw(image)  

    colors = ('#FF0000','#0000FF','#FF00FF','#F1C40F')
    dict1 = {'packer':0,'stabilizer':1,'joint':2,'wrench':3}  
    # calculate and display bounding boxes for each detected custom label       
    for customLabel in response['CustomLabels']:
        if customLabel['Confidence'] >= confidence:
            if (customLabel['Name']== 'stabilizer' and customLabel['Confidence']<100):
                continue
            
            if 'Geometry' in customLabel:
                box = customLabel['Geometry']['BoundingBox']
                left = imgWidth * box['Left']
                top = imgHeight * box['Top']
                width = imgWidth * box['Width']
                height = imgHeight * box['Height']
                draw.text((left + width-20,top-15), customLabel['Name'], fill= colors[dict1[customLabel['Name']]])
                draw.text((left,top-15), str(customLabel['Confidence'])[:6], fill= colors[dict1[customLabel['Name']]])#, font='Arial.ttf') 

                points = (
                    (left,top),
                    (left + width, top),
                    (left + width, top + height),
                    (left , top + height),
                    (left, top))
                draw.line(points, fill= colors[dict1[customLabel['Name']]], width=5)

    image.show()
    display(image)
    image = image.convert('RGB')
    image.save('inference.jpg',optimize=True,quality=300)

    return len(response['CustomLabels'])

## 3- Detect Tools (Editing required)

In [ ]:
bucket="your bucket name where you stored your images"
model= "Your model arn. It should be similar to below exapmple"
photo = 'path to picture in your bucket'


# Example:

# bucket = 'bucket-name'
# model = "arn:aws:rekognition:us-east-1:745137163025:project/cop/version/cop.2020-01-31T12.17.29/1580494650055"
# photo = 'pictures/4.png'

label_count=show_custom_labels(model,bucket,photo)

## 4- Stop your Rekognition model (Editing required). After pasting remove the following line: --min-inference-units 1 \

In [ ]:
# After pasting remove the following line: --min-inference-units 1 \

!aws rekognition stop-project-version \
  --project-version-arn "arn:aws:rekognition:us-east-1:745137163025:project/cop/version/cop.2020-01-31T12.17.29/1580494650055" \
  --min-inference-units 1 \
  --endpoint-url https://rekognition.us-east-1.amazonaws.com \
  --region us-east-1